<a href="https://colab.research.google.com/github/Byeon-MJ/Dacon_Repo/blob/main/Dacon_Synthetic_data_Object_detection/Dacon_Synthetic_data_Object_detection_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Synthetic data Object detection Baseline Code
https://dacon.io/competitions/official/236107/overview/description

Use Pytorch & Faster-RCNN

## Import

In [4]:
import warnings
warnings.filterwarnings(action='ignore')

In [5]:
import random
import pandas as pd
import numpy as np
import os
import glob
import cv2
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

import torchvision
import torchvision.models as models
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor, FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone

import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import cv2
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from tqdm.auto import tqdm

In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')